In [2]:
%load_ext autoreload
%autoreload 2
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score
from answer_judge import load_judge

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
data_train=ReadData('/root/autodl-tmp/hallscope/AOKVQA/train').read_all()
random.shuffle(data_train)

In [4]:
df_train=pd.DataFrame(data_train)

In [5]:
df_train.head()

,question,answer,most_likely,responses
0,[Question:Which one of these foods is most lik...,[tensor(0)],{'embedding': [[[-0.00982666 0.00026894 0.00...,None
1,[Question:Why is he standing like that?\nAnswe...,[tensor(2)],{'embedding': [[[ 0.01708984 0.01708984 0.00...,None
2,[Question:Why is the horse in the town center?...,[tensor(2)],{'embedding': [[[-0.00427246 0.01159668 -0.02...,None
3,[Question:What kind of candy bar is hanging on...,[tensor(1)],{'embedding': [[[-0.00982666 0.00026894 0.00...,None
4,[Question:What is this image?\nAnswer the Ques...,[tensor(3)],{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None


In [6]:
i=0
df_train.iloc[i]['most_likely']['response']

['A. sushi']

In [8]:
df_train.iloc[i]['question']

['Question:Which one of these foods is most likely to be served by the waitress?\nAnswer the Question with following choices.\nChoices: A.sushi B.pierogi C.taco D.roti\n']

In [9]:
df_train.iloc[0]['answer']

tensor([0])

In [10]:
embeddings=df_train['most_likely'].apply(lambda x: x['embedding']).to_list()
embeddings=np.concatenate(embeddings,axis=0)
embeddings.shape

(17056, 29, 3584)

In [11]:
gt_label=df_train['answer'].to_list()
gt_label=np.concatenate(gt_label,axis=0)
gt_label.shape

(17056,)

In [12]:
gt_label[0]

np.int64(0)

In [13]:
judge=load_judge('OKVQA')

In [14]:
responses=df_train['most_likely'].apply(lambda x: x['response']).to_list()

rsps=[]
for x in responses:
    rsps+=x
    
is_correct=[]
for i,x in enumerate(rsps):
    is_correct.append(judge.check(x,gt_label[i]))

is_correct=np.array(is_correct)
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 17056 responses. 14218 answer correctly and 2838 answer wrongly


In [41]:
length = len(embeddings)
train_rate = 0.8
num_val=3000

embedding_train = embeddings[:int(length*train_rate)-num_val]
embedding_val = embeddings[int(length*train_rate)-num_val:int(length*train_rate)]
embedding_test = embeddings[int(length*train_rate):]

hall_label_train = is_correct[:int(length*train_rate)-num_val]
hall_label_val = is_correct[int(length*train_rate)-num_val:int(length*train_rate)]
hall_label_test = is_correct[int(length*train_rate):]


'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
print(f'trainset length:{len(hall_label_train)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(
    f'trainset postive: {sum(hall_label_train==1)} negative: {sum(hall_label_train==0)}')
print(
    f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:10644
validset length:3000
testset length:3412
trainset postive: 8868 negative: 1776
valset postive: 2479 negative: 521
testset postive: 2871 negative: 541


In [42]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCALinear,PCAKernel,KernelPCA
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score,balanced_accuracy_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [43]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCALinear(X[:,i_layer,:],n_components)
    scores=discriminator.get_score(X[:,i_layer,:])
    
    return roc_auc_score(y,scores)
    #split=discriminator.get_best_split(scores,y)
    #y_pred=scores>split
    #return accuracy_score(y,y_pred)
    #return balanced_accuracy_score(y,y_pred)
    #
    #split=discriminator.get_best_split(y)
    #y_preds=(scores>split)
    #return accuracy_score(y,y_preds)

In [44]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,4),
    'layer':range(10,embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

found best n_components:1,layer:10 Score: 0.5313973267965304
found best n_components:1,layer:12 Score: 0.5320171203948096
found best n_components:1,layer:17 Score: 0.5339554755144751
found best n_components:1,layer:28 Score: 0.5609058509909343


  0%|          | 0/19 [00:00<?, ?it/s]

found best n_components:2,layer:19 Score: 0.5680774165175575


  0%|          | 0/19 [00:00<?, ?it/s]

found best n_components:3,layer:19 Score: 0.5690053648342817


In [25]:
best_paras

{'n_components': 2, 'layer': 19, 'best_score': np.float64(0.5697979919489671)}

In [26]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [27]:
discriminator=PCALinear(embedding_val[:,best_layer,:],best_paras['n_components'])
scores=discriminator.get_score(embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(scores,hall_label_val)

In [29]:
discriminator=PCALinear(embedding_train[:,best_layer,:],best_paras['n_components'])
scores=discriminator.get_score(embedding_train[:,best_layer,:])
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_train,y_train)} AUROC:{roc_auc_score(hall_label_train,scores)}')

train set acc: 0.8084850566815527 AUROC:0.5697374796993118


In [30]:
discriminator=PCALinear(embedding_test[:,best_layer,:],best_paras['n_components'])
scores=discriminator.get_score(embedding_test[:,best_layer,:])
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.5896834701055099 AUROC:0.5352363587432744


Train Linear Prob to Get Better Results

In [31]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [32]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [35]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} f1:{f1_score(y_test,pred_test)} auroc:{roc_auc_score(y_test,scores_test)}  b-acc:{balanced_accuracy_score(y_test,pred_test)}')

test set acc: 0.8147713950762017 f1:0.8972357723577236 auroc:0.5500495425283494  b-acc:0.49990310395689963
